In [ ]:
import os
import cv2
import glob

In [ ]:
def getFrame(sec, vidcap, count):
    vidcap.set(cv2.CAP_PROP_POS_MSEC,sec*1000)
    hasFrames, image = vidcap.read()

    if hasFrames:
        #image = cv2.resize(image, dsize=(120,120))              #resize image to 120x120
        filename = '/content/drive/MyDrive/Testing/frames_test/' + "%d.png" % count
        cv2.imwrite(filename, image)    # save frame as JPG file
        print(filename)

    return hasFrames

In [ ]:
def createFrames(video):
  
  #first remove the existing frames from directory
  print('removing frames...')
  path = '/content/drive/MyDrive/Testing/frames_test'
  files = os.listdir(path)
  for f in files:
    os.remove(path+'/'+f)
    #print(f)
  
  print('creating frames...')

  vidcap = cv2.VideoCapture(video)
  sec = 0
  frameRate = 1/6  # it will capture image in each 1/6th second
  count = 1 
  success = getFrame(sec, vidcap, count)

  while success:
    count = count + 1
    sec = sec + frameRate
    sec = round(sec, 2)
    success = getFrame(sec, vidcap, count)

In [ ]:
#global sentences dictionary
sentences_dict = {0: 'None', 1:'آج کیا تاریخ ہے', 2:'پولیس سکول میں جا رہی ہے', 3:'پاکستان میرا پیارا وطن ہے', 4:'واش روم کدھر ہے', 5:'میری دو بہنیں ہیں',
                  6:'میرے دو بھائ ہیں', 7:'میں طالب علم ہوں', 8:'مجھے راستہ بھول گیا ہے', 9:'کیا وقت ہوا ہے', 10:'بلی کو بھوک لگی ہے',
                  11:'اس بچے کو پانی چاہیے', 12:'آپ کا شکریہ', 13:'آپ سے مل کر اچھا لگا', 14:'مجھے میرے امی ابو سے پیار ہے',
                  15:'میرے دوست کو چاکلیٹ پسند ہے', 16:'میری بہن نے ٹرین نہیں دیکھی', 17:'براہ مہربانی مجھے جانے دیں',
                  18:'بھائ کمپیوٹر خریدنے کے لیے پیسے دیں', 19:'ہم ملتان سے لاھور ٹرین میں گۓ', 20:'مجھے اب جانا چاہیے'}

In [ ]:
import numpy as np
import os
import cv2
import keras
from keras.models import Sequential,Input,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from sklearn.model_selection import KFold

In [ ]:
def toNumpy(dir):
    img_array = np.empty([0,43200], np.uint8, 'C')
    print('Converting frames to Numpy array...')

    dirs = os.listdir(dir)
    for img in dirs:
      #print(img)
      path = dir+"/"+img

      if path.endswith(".png"):
        I = cv2.imread(path)
        I = cv2.resize(I, dsize=(120,120))
        im = (np.array(I))
        r = im[:, :, 0].flatten()
        g = im[:, :, 1].flatten()
        b = im[:, :, 2].flatten()
        a = [list(r) + list(g) + list(b)]
        b = np.asarray(a)
        img_array = np.append(img_array, b, axis=0)
    
    print('Reshaping array...')
    img_array = img_array.reshape(img_array.shape[0], 120, 120, 3)
    print('Normalizing array...')
    img_array = img_array.astype('float32')
    img_array = img_array/ 255.

    return img_array

In [ ]:
def predictSentence(frames_dir):

  #preprocess frames
  images_array = toNumpy(frames_dir)

  #define CNN model
  sign_model = Sequential()
  sign_model.add(Conv2D(32, kernel_size=(3, 3),activation='linear',padding='same',input_shape=(120,120,3)))
  sign_model.add(LeakyReLU(alpha=0.1))
  sign_model.add(MaxPooling2D((2, 2),padding='same'))
  sign_model.add(Dropout(0.4))
  sign_model.add(Conv2D(64, (3, 3), activation='linear',padding='same'))
  sign_model.add(LeakyReLU(alpha=0.1))
  sign_model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
  sign_model.add(Dropout(0.4))
  sign_model.add(Conv2D(128, (3, 3), activation='linear',padding='same'))
  sign_model.add(LeakyReLU(alpha=0.1))                  
  sign_model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
  sign_model.add(Dropout(0.4))
  sign_model.add(Flatten())
  sign_model.add(Dense(128, activation='linear'))
  sign_model.add(LeakyReLU(alpha=0.1))           
  sign_model.add(Dropout(0.4))
  sign_model.add(Dense(21, activation='softmax'))

  # loading the trained weights
  sign_model.load_weights("/content/drive/MyDrive/PSL dataset/CNN_bg_weights.hdf5")

  # compiling the model
  sign_model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(),metrics=['accuracy'])
  
  #predict the video
  print('Pridicting Sentence...')
  predictions = sign_model.predict(images_array)
  predictions = np.argmax(np.round(predictions),axis=1)
  print(predictions)

  #maximum predicted sentence
  sentence_key = np.bincount(predictions).argmax()
  occurrences = np.count_nonzero(predictions == sentence_key)
  print('confidence:', occurrences/len(predictions))

  #take sentence from dictionary using predicted key
  global sentences_dict
  sentence = sentences_dict[sentence_key]

  return 'Predicted sentence is: '+sentence

In [ ]:
createFrames('/content/drive/MyDrive/PSL Schools Dataset/front/Farukh/sentence18.mp4')

removing frames...
creating frames...
/content/drive/MyDrive/Testing/frames_test/1.png
/content/drive/MyDrive/Testing/frames_test/2.png
/content/drive/MyDrive/Testing/frames_test/3.png
/content/drive/MyDrive/Testing/frames_test/4.png
/content/drive/MyDrive/Testing/frames_test/5.png
/content/drive/MyDrive/Testing/frames_test/6.png
/content/drive/MyDrive/Testing/frames_test/7.png
/content/drive/MyDrive/Testing/frames_test/8.png
/content/drive/MyDrive/Testing/frames_test/9.png
/content/drive/MyDrive/Testing/frames_test/10.png
/content/drive/MyDrive/Testing/frames_test/11.png
/content/drive/MyDrive/Testing/frames_test/12.png
/content/drive/MyDrive/Testing/frames_test/13.png
/content/drive/MyDrive/Testing/frames_test/14.png
/content/drive/MyDrive/Testing/frames_test/15.png
/content/drive/MyDrive/Testing/frames_test/16.png
/content/drive/MyDrive/Testing/frames_test/17.png
/content/drive/MyDrive/Testing/frames_test/18.png
/content/drive/MyDrive/Testing/frames_test/19.png
/content/drive/MyDriv

In [ ]:
#for 90 degree clockwise rotation
path = '/content/drive/MyDrive/Testing/frames_test'
files = os.listdir(path)
for f in files:
  src = cv2.imread(path+'/'+f)
  
  # Using cv2.rotate() method
  # Using cv2.ROTATE_90_CLOCKWISE rotate
  # by 90 degrees clockwise
  image = cv2.rotate(src, cv2.cv2.ROTATE_90_CLOCKWISE)
  cv2.imwrite(path+'/'+f, image)

In [ ]:
print(predictSentence('/content/drive/MyDrive/Testing/frames_test'))

Converting frames to Numpy array...
Reshaping array...
Normalizing array...
Pridicting Sentence...
[16 16 16 16 16 16 16 16 16 17  0  0  0 17 17 17 17 17 17 16 16 16 16 16
 16 17 16 16  0 15 15  0  0 16 16]
confidence: 0.5428571428571428
Predicted sentence is: میری بہن نے ٹرین نہیں دیکھی


In [ ]:
def predictSentence(frames_dir):

  #preprocess frames
  images_array = toNumpy(frames_dir)

  #define CNN model (testtest1)
  sign_model = Sequential()
  sign_model.add(Conv2D(64, kernel_size=(5, 5),activation='linear',padding='same',input_shape=(120,120,3)))
  sign_model.add(LeakyReLU(alpha=0.1))
  sign_model.add(MaxPooling2D((2, 2),padding='same'))
  #sign_model.add(Dropout(0.5))
  sign_model.add(Conv2D(64, (5, 5), activation='linear',padding='same'))
  sign_model.add(LeakyReLU(alpha=0.1))
  sign_model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
  #sign_model.add(Dropout(0.5))
  sign_model.add(Conv2D(64, (5, 5), activation='linear',padding='same'))
  sign_model.add(LeakyReLU(alpha=0.1))                  
  sign_model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
  #sign_model.add(Dropout(0.5))
  sign_model.add(Flatten())
  sign_model.add(Dense(256, activation='linear'))
  sign_model.add(LeakyReLU(alpha=0.1))           
  #sign_model.add(Dropout(0.5))
  sign_model.add(Dense(128, activation='linear'))
  sign_model.add(LeakyReLU(alpha=0.1))           
  #sign_model.add(Dropout(0.5))
  sign_model.add(Dense(21, activation='softmax'))

#epochs 735, accuracy 93.4, lr 1e-5

  # loading the trained weights
  sign_model.load_weights("/content/drive/MyDrive/PSL dataset/best_model.hdf5")

  # compiling the model
  sign_model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(),metrics=['accuracy'])
  
  #predict the video
  print('Pridicting Sentence...')
  predictions = sign_model.predict(images_array)
  predictions = np.argmax(np.round(predictions),axis=1)
  print(predictions)

  #maximum predicted sentence
  sentence_key = np.bincount(predictions).argmax()
  occurrences = np.count_nonzero(predictions == sentence_key)
  print('confidence:', occurrences/len(predictions))

  #take sentence from dictionary using predicted key
  global sentences_dict
  sentence = sentences_dict[sentence_key]

  return 'Predicted sentence is: '+sentence